## 算术运算

### 数据对齐，填充值
不同索引的对象之间可以进行算术运算。
- 若存在不同的索引项，运算结果的索引是运算对象的索引的并集，而在两个对象中并不同时存在的索引处值为空值NaN；
- 一个运算对象使用add方法传入另外一个运算对象和fill_value参数，运算对象在另外一个对象中不存在的索引处值填入参数fill_value的值然后再进行运算；
- 类似add方法，还有sub、div、floordiv、mul，pow方法，若在前面加上字母“r”，则表示翻转参数，比如被减数和减数身份对换；
- fill_value参数同样适用于reindex重新索引；

In [1]:
import numpy as np
import pandas as pd
frame1=pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]],columns=list('ABC'),index=list('bce'))
frame2=pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]],columns=list('ACD'),index=list('abc'))

In [2]:
frame1

,A,B,C
b,1,2,3
c,4,5,6
e,7,8,9


In [3]:
frame2

,A,C,D
a,1,2,3
b,4,5,6
c,7,8,9


In [4]:
# 相加后，行索引和列索引为frame1和frame2的行、列索引的并集
# 在frame1和frame2同时存在的索引位置的数值为运算结果，其他均为NaN
frame1+frame2

,A,B,C,D
a,NaN,NaN,NaN,NaN
b,5.0,NaN,8.0,NaN
c,11.0,NaN,14.0,NaN
e,NaN,NaN,NaN,NaN


In [5]:
frame1.add(frame2,fill_value=0) # 使用add方法，可以在不存在的索引处填入值

,A,B,C,D
a,1.0,NaN,2.0,3.0
b,5.0,2.0,8.0,6.0
c,11.0,5.0,14.0,9.0
e,7.0,8.0,9.0,NaN


In [6]:
frame1.sub(frame2,fill_value=0) # 减法，frame1-frame2

,A,B,C,D
a,-1.0,NaN,-2.0,-3.0
b,-3.0,2.0,-2.0,-6.0
c,-3.0,5.0,-2.0,-9.0
e,7.0,8.0,9.0,NaN


In [7]:
frame1.rsub(frame2,fill_value=0) # rsub，翻转参数，frame2-frame1

,A,B,C,D
a,1.0,NaN,2.0,3.0
b,3.0,-2.0,2.0,6.0
c,3.0,-5.0,2.0,9.0
e,-7.0,-8.0,-9.0,NaN


In [8]:
frame1.reindex(index=frame2.index) # 使用frame2的行索引对frame1重新索引，缺失内容为空

,A,B,C
a,NaN,NaN,NaN
b,1.0,2.0,3.0
c,4.0,5.0,6.0


In [9]:
frame1.reindex(index=frame2.index,fill_value=0) # reindex也可以使用fill_value参数填充缺失内容

,A,B,C
a,0,0,0
b,1,2,3
c,4,5,6


### DataFrame和Series之间运算
- DataFrame和Series之间运算，将Series的索引匹配到DataFrame的列，并一行行向下广播；
- 若有缺失的索引，会重新索引生成索引的并集；
- 若需要将Series索引匹配到DataFrame的行，并一在列上广播，必须使用算术运算方法，且在参数里指定轴为行索引axis='index'或axis=0，算术方法前加字母“r”可以翻转参数；

In [10]:
frame1

,A,B,C
b,1,2,3
c,4,5,6
e,7,8,9


In [11]:
s1=frame1.loc['b'] # 选取frame1中标签为b的行作为s1
s1

A    1
B    2
C    3
Name: b, dtype: int64

In [12]:
frame1-s1 # s1的索引和frame1的列匹配，并逐行进行运算

,A,B,C
b,0,0,0
c,3,3,3
e,6,6,6


In [13]:
s2=s1.copy() # 使用s1副本生成s2
s2.index=list('BAD') # 对s2重新索引，使之存在frame1中没有的索引项，并改变已存在的索引项顺序
s2

B    1
A    2
D    3
Name: b, dtype: int64

In [14]:
frame1-s2 # 根据对应的索引标签值来运算而非位置，缺失的索引项被加入索引之中，且值为空

,A,B,C,D
b,-1.0,1.0,NaN,NaN
c,2.0,4.0,NaN,NaN
e,5.0,7.0,NaN,NaN


In [15]:
s3=frame1['B'] # 获取frame1的B列作为s3
s3

b    2
c    5
e    8
Name: B, dtype: int64

In [16]:
frame1.sub(s3,axis='index') # 匹配行，一列一列进行运算，必须使用算术方法，参数指定轴为行索引index

,A,B,C
b,-1,0,1
c,-1,0,1
e,-1,0,1


In [17]:
frame1.rsub(s3,axis='index') # 同样方法名前面加上“r”可以翻转参数

,A,B,C
b,1,0,-1
c,1,0,-1
e,1,0,-1
